In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
df_House_A = pd.read_csv('House_A.csv', index_col = 'time_stamp', parse_dates=True)
df_House_A.index.freq = 'S'

df_House_B = pd.read_csv('House_B.csv', index_col = 'time_stamp', parse_dates=True)
df_House_B.index.freq = 'S'

In [2]:
print(df_House_A.shape)
print(df_House_B.shape)

(2592000, 25)
(2592000, 25)


In [21]:
number_points_predict = 30000
training_datapoints = 1000000
df_House_A_train = df_House_A[training_datapoints:training_datapoints*2]
df_House_A_test = df_House_A[training_datapoints*2:]

df_House_B_train = df_House_B[training_datapoints:training_datapoints*2]
df_House_B_test = df_House_B[training_datapoints*2:]

def predict_labels(df, steps):
    # Define the order of the ARIMA model
    order = (1, 1, 0)  # (p, d, q)

    # Fit the ARIMA model to Resident_1 column
    model_1 = ARIMA(df['Resident_1'], order=order)
    result_1 = model_1.fit()

    # Make a forecast for Resident_1 column
    forecast_1 = result_1.forecast(steps=steps)

    # Fit the SARIMA model to Resident_2 column
    model_2 = ARIMA(df['Resident_2'], order=order)
    result_2 = model_2.fit()

    # Make a forecast for Resident_2 column
    forecast_2 = result_2.forecast(steps=steps)
    return forecast_1, forecast_2

forecast_A1, forecast_A2 = predict_labels(df_House_A_train, number_points_predict)

forecast_B1, forecast_B2 = predict_labels(df_House_B_train, number_points_predict)




In [24]:
from sklearn.metrics import mean_squared_error

# Load the actual values for Resident_1 and Resident_2
actual_A1 = df_House_A_test['Resident_1'][:number_points_predict]
actual_A2 = df_House_A_test['Resident_2'][:number_points_predict]

forecast_A1 = forecast_A1.apply(lambda x: int(x))
forecast_A2 = forecast_A2.apply(lambda x: int(x))
df_House_A_test['Resident_1_Pred'] = forecast_A1
df_House_A_test['Resident_2_Pred'] = forecast_A2


# Calculate the mean squared error for Resident_1
actual_B1 = df_House_B_test['Resident_1'][:number_points_predict]
actual_B2 = df_House_B_test['Resident_2'][:number_points_predict]
forecast_B1 = forecast_B1.apply(lambda x: int(x))
forecast_B2 = forecast_B2.apply(lambda x: int(x))
df_House_B_test['Resident_1_Pred'] = forecast_B1
df_House_B_test['Resident_2_Pred'] = forecast_B2



print()
#Calculate the mean squared error for Resident_1 House A
mse_A1 = mean_squared_error(actual_A1, forecast_A1)
print('House A - Resident_1 MSE:', mse_A1)
# Calculate the mean squared error for Resident_2
mse_A2 = mean_squared_error(actual_A2, forecast_A2)
print('House A - Resident_1 MSE:', mse_A2)
mse_B1 = mean_squared_error(actual_B1, forecast_B1)
print('House B - Resident_1 MSE:', mse_B1)
mse_B2 = mean_squared_error(actual_B2, forecast_B2)
print('House B - Resident_2 MSE:', mse_B2)

#Save dataframes for tableau dashboard
df_House_B_test[['Resident_1', 'Resident_2', 'Resident_1_Pred','Resident_2_Pred']].head(number_points_predict).to_csv("House_B_TS_Pred.csv")
df_House_A_test[['Resident_1', 'Resident_2', 'Resident_1_Pred','Resident_2_Pred']].head(number_points_predict).to_csv("House_A_TS_Pred.csv")

<ipython-input-24-fc9e8fd4d8f1>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_House_A_test['Resident_1_Pred'] = forecast_A1
<ipython-input-24-fc9e8fd4d8f1>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_House_A_test['Resident_2_Pred'] = forecast_A2
<ipython-input-24-fc9e8fd4d8f1>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

House A - Resident_1 MSE: 10.574133333333334
House A - Resident_1 MSE: 41.806066666666666
House B - Resident_1 MSE: 7.2584
House B - Resident_2 MSE: 5.1661
